# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.  

#### Необходимо

Обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении имеется набор данных с разметкой о токсичности правок.
Построить модель со значением метрики качества F1 не меньше 0.75.

In [1]:
!pip install pymystem3

Подключим необхоимые библиотеки

In [2]:
import numpy as np
import pandas as pd
import nltk
import re
from pymystem3 import Mystem
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from tqdm import notebook
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# загружаем класс pipeline
from sklearn.pipeline import Pipeline

# импортируем класс GridSearchCV
from sklearn.model_selection import GridSearchCV

# загружаем нужные модели
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/iuser24/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/iuser24/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/iuser24/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/iuser24/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Загрузим датасет с комметариями

In [4]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col='Unnamed: 0')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Датасет содержит 159292 записей комментариев с пометкой токсичности

In [6]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Язык комментариев - английский

In [7]:
lemmatizer = WordNetLemmatizer()

In [8]:
def lemmatize_text(text):
    # Переведем текст в нижний регистр
    text = text.lower()
    # Разложим на токены
    tokens = word_tokenize(text)
    # Приведем к каждому токену часть речи
    pos_tags = pos_tag(tokens)
    # Произведем лемматизацию в соответствии с частью речи
    lemmatized_tokens = [
        lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in pos_tags
    ]
    # Очистим текст от цифр и знаков препинания
    cleared_text = clear_text(" ".join(lemmatized_tokens))
    return cleared_text

In [9]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [10]:
def clear_text(text):
    clr_txt = re.sub(r'[^a-zA-Z]', ' ', text)
    clr_txt_list = clr_txt.split() 
    return ' '.join(clr_txt_list)

Произведем обработку текста

In [11]:
df['lemm_text'] = df['text'].apply(lemmatize_text)

In [12]:
# Удалим исходный стобец
df = df.drop(['text'], axis=1)

In [13]:
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/iuser24/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Разделим датасет на тренировочную и тестовую выборки

In [14]:
train, test = train_test_split(df, test_size=0.1)

Векторизируем текст

In [15]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(train['lemm_text'])
tf_idf_test = count_tf_idf.transform(test['lemm_text'])

In [16]:
print("Размер матрицы train:", tf_idf_train.shape)
print("Размер матрицы test:", tf_idf_test.shape)

Размер матрицы train: (143362, 148617)
Размер матрицы test: (15930, 148617)


Создадим пайплайн для обучения двух моделей: Логистическая регрессия и Градиентный бустинг

In [17]:
RANDOM_STATE = 42

In [18]:
pipe_final = Pipeline([
    ('models', LogisticRegression(random_state=RANDOM_STATE))
])

In [19]:
param_grid = [
    {
        'models': [
            LogisticRegression(random_state=RANDOM_STATE),
            lgb.LGBMClassifier()
        ]
    },
]

Запустим обучение и поиск лучшей модели. В качестве метрики выберем Accuracy. Она покажет нам сколько правильных предсказаний сделала модель относительно общего числа предсказаний.

In [20]:
searchcv = GridSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
searchcv.fit(tf_idf_train, train['toxic'])

[LightGBM] [Info] Number of positive: 14530, number of negative: 128832
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 11.725938 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 594330
[LightGBM] [Info] Number of data points in the train set: 143362, number of used features: 10936
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.101352 -> initscore=-2.182294
[LightGBM] [Info] Start training from score -2.182294


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('models',
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'models': [LogisticRegression(random_state=42),
                                     LGBMClassifier()]}],
             scoring='accuracy')

In [21]:
print('Лучшая модель и её параметры:\n\n', searchcv.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', searchcv.best_score_)

Лучшая модель и её параметры:

 Pipeline(steps=[('models', LGBMClassifier())])
Метрика лучшей модели на тренировочной выборке: 0.9556925839091249


In [22]:
best_model = searchcv.best_estimator_.named_steps['models']

In [23]:
toxic_pred = best_model.predict(tf_idf_test)

In [25]:
print('Точность предсказаний на тестовой выборке составляет:', accuracy_score(test['toxic'], toxic_pred))

Точность предсказаний на тестовой выборке составляет: 0.9531701192718142


## Вывод

В ходе работы 159292 записей комментариев с пометкой токсичности. Комментарии были обработаны. Обработка заключала в себе следующие шаги:

- Перевод текста в нижний регистр
- Лемматизация в соответствии с частью речи
- Очистка текста от цифр и знаков препинания

Далее текст был векторизирован и разделен на тренировочную и тестовую выборки. Модели для обучения были выбраны логистическая регрессия и градиентный бустинг. В качестве метрики выбран Accuracy. Данная метрика показала нам сколько правильных предсказаний сделала модель относительно общего числа предсказаний. В результате обучения была наилучшие результаты показала модель градиентного бустинга. Результат у лучшей модели на тестовой выборкесоставил 95%. Это говорит о том, что модель в 95-ти случаях из 100 успешно определяет токсичный комментарий.